Figure: radar plot of distribution of body regions

In [ ]:
import pandas as pd
from utils.utils import get_multi_label_counts

In [ ]:
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/data/trauma_categories_Rega Pain Study15.09.2025_v2.xlsx'
medic_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/data/Liste Notärzte-1.xlsx'

In [ ]:
restrict_to_trauma = True
restrict_to_primary = True

In [ ]:
data_df = pd.read_excel(data_path)
medic_df = pd.read_excel(medic_data_path)

In [ ]:
duplicates = data_df[data_df["SNZ Ereignis Nr. "].duplicated()]["SNZ Ereignis Nr. "]
print(f'Duplicates found: {duplicates.nunique()}')
# drop duplicates
data_df = data_df.drop_duplicates(subset=["SNZ Ereignis Nr. "])

In [ ]:
data_df = data_df[data_df["VAS_on_scene"] > 3]

if restrict_to_trauma:
    n_non_trauma = data_df[data_df['Einteilung (reduziert)'] != 'Unfall'].shape[0]
    print(f'Excluded {n_non_trauma} non-trauma patients')

    # adult non-trauma patients
    n_adult_non_trauma = data_df[(data_df['Einteilung (reduziert)'] != 'Unfall') & (data_df["Alter "] >= 16)].shape[0]
    print(f'Excluded {n_adult_non_trauma} adult non-trauma patients')
    # pediatric non-trauma patients
    n_pediatric_non_trauma = data_df[(data_df['Einteilung (reduziert)'] != 'Unfall') & (data_df["Alter "] < 16)].shape[0]
    print(f'Excluded {n_pediatric_non_trauma} pediatric non-trauma patients')

    data_df = data_df[data_df['Einteilung (reduziert)'] == 'Unfall']

if restrict_to_primary:
    n_secondary = data_df[data_df['Einsatzart'] != 'Primär'].shape[0]
    print(f'Excluded {n_secondary} secondary transport patients')

    # adult secondary transport patients
    n_adult_secondary = data_df[(data_df['Einsatzart'] != 'Primär') & (data_df["Alter "] >= 16)].shape[0]
    print(f'Excluded {n_adult_secondary} adult secondary transport patients')
    # pediatric secondary transport patients
    n_pediatric_secondary = data_df[(data_df['Einsatzart'] != 'Primär') & (data_df["Alter "] < 16)].shape[0]
    print(f'Excluded {n_pediatric_secondary} pediatric secondary transport patients')
    data_df = data_df[data_df['Einsatzart'] == 'Primär']

In [ ]:
adult_df = data_df[data_df["Alter "] >= 16]

In [ ]:
def preprocess_body_region(df):
    # fill na in Körperregion with Körperregion2
    df['Körperregion'] = df['Körperregion'].fillna(df['Körperregion2'])
    # set to lower
    df['Körperregion'] = df['Körperregion'].str.lower()
    # replace kopf and 'schädel-hirn/hals' and 'schädel-hirn/gesicht' and 'schädel-hirn/hals' with schädel-hirn
    df['Körperregion'] = df['Körperregion'].str.replace('kopf', 'schädel-hirn')
    df['Körperregion'] = df['Körperregion'].str.replace('schädel-hirn/hals', 'schädel-hirn')
    df['Körperregion'] = df['Körperregion'].str.replace('schädel-hirn/gesicht', 'schädel-hirn')
    df['Körperregion'] = df['Körperregion'].str.replace('schädel-hirn/hals', 'schädel-hirn')
    # replace 'kopf/gesicht' / 'augen' / 'kopf/hals' / 'kopf (gehör)' / 'kopf/gesicht/hals'  and 'schädel-hirn (gehör)' with 'gesicht'
    # df['Körperregion'] = df['Körperregion'].str.replace({'kopf/gesicht': 'gesicht', 'augen': 'gesicht', 'kopf/hals': 'gesicht', 'kopf (gehör)': 'gesicht', 'kopf/gesicht/hals': 'gesicht'})
    df['Körperregion'] = df['Körperregion'].str.replace('kopf/gesicht', 'gesicht')
    df['Körperregion'] = df['Körperregion'].str.replace('kopf/hals', 'gesicht')
    df['Körperregion'] = df['Körperregion'].str.replace('kopf (gehör)', 'gesicht')
    df['Körperregion'] = df['Körperregion'].str.replace('kopf/gesicht/hals', 'gesicht')
    df['Körperregion'] = df['Körperregion'].str.replace('augen', 'gesicht')
    df['Körperregion'] = df['Körperregion'].str.replace('schädel-hirn (gehör)', 'gesicht')
    # replace 'rücken' with 'wirbelsäule'
    df['Körperregion'] = df['Körperregion'].str.replace('rücken', 'wirbelsäule')
    # replace 'rump' with 'thorax'
    df['Körperregion'] = df['Körperregion'].str.replace('rumpf', 'thorax')
    # replace  'obere extremiät' and 'obere extermität' with 'obere extremität'    
    df['Körperregion'] = df['Körperregion'].str.replace('obere extremiät', 'obere extremität')
    df['Körperregion'] = df['Körperregion'].str.replace('obere extermität', 'obere extremität')
    # replace 'untere extermität' with 'untere extremität'    
    df['Körperregion'] = df['Körperregion'].str.replace('untere extermität', 'untere extremität')
    # replace '' with pd.NA
    df['Körperregion'] = df['Körperregion'].replace('', pd.NA)

    return df

In [ ]:
adult_df = preprocess_body_region(adult_df)

In [ ]:
dx_counts = get_multi_label_counts(adult_df, 'Körperregion')

In [ ]:
# remove the '' entry from dx_counts
dx_counts.pop('', None)

In [ ]:
print("dx_counts:", dx_counts)
print(f"Total adult patients: {len(adult_df)}")
print(f"Number of body regions: {len(dx_counts)}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from math import pi

def create_radar_chart(dx_counts, total_patients, title="Body Region Distribution", 
                       figsize=(10, 10), min_percentage_threshold=0.5):
    """
    Create a radar chart from dx_counts object showing percentages of patients
    
    Parameters:
    -----------
    dx_counts : dict
        Dictionary with body regions as keys and counts as values
    total_patients : int
        Total number of patients for percentage calculation
    title : str
        Title for the chart
    figsize : tuple
        Figure size (width, height)
    min_percentage_threshold : float
        Minimum percentage to include in chart (filters out very small categories)
    
    Returns:
    --------
    fig, ax : matplotlib figure and axis objects
    """
    
    # Create label mapping for English display
    label_mapping = {
        'untere extremität': 'Lower extremity',
        'obere extremität': 'Upper extremity', 
        'wirbelsäule': 'Spine',
        'thorax': 'Thorax',
        'schädel-hirn': 'Head',
        'becken': 'Pelvis',
        'abdomen': 'Abdomen',
        'gesicht': 'Face',
        'hals': 'Neck',
        'polytrauma': 'Polytrauma',
        'systemisch': 'Systemic',
        'weichteile (genitalien)': 'Soft Tissue (Genitals)',
        'keine': 'None'
    }
    
    # Filter out categories below threshold and convert counts to percentages
    filtered_data = {}
    filtered_counts = {}
    for region, count in dx_counts.items():
        percentage = (count / total_patients) * 100
        if percentage >= min_percentage_threshold:
            filtered_data[region] = percentage
            filtered_counts[region] = count
    
    # Sort by percentage (descending)
    sorted_data = dict(sorted(filtered_data.items(), key=lambda x: x[1], reverse=True))
    
    # Prepare data for radar chart
    categories = list(sorted_data.keys())
    display_labels = []
    for cat in categories:
        clean_label = label_mapping.get(cat, cat.title())
        count = filtered_counts[cat]
        display_labels.append(f"{clean_label} ({count/total_patients*100:.1f}%)")
    
    values = list(sorted_data.values())
    
    # Number of variables
    N = len(categories)
    
    # Compute angles for each category
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]  # Complete the circle
    
    # Add first value to end to complete the circle
    values += values[:1]
    
    # Create the plot
    fig, ax = plt.subplots(figsize=figsize, subplot_kw=dict(projection='polar'))
    
    # Plot the values with clean styling like the reference image
    ax.plot(angles, values, 'o-', linewidth=2, markersize=6, 
            color='#1f77b4', markerfacecolor='#1f77b4', markeredgecolor='#1f77b4')
    
    # Fill the area with light blue similar to reference
    ax.fill(angles, values, alpha=0.25, color='#1f77b4')
    
    # Set clean white background
    ax.set_facecolor('white')
    fig.patch.set_facecolor('white')
    
    # Add category labels positioned outside the chart
    ax.set_xticks(angles[:-1])
    
    # Position labels outside the chart area like in the reference image
    max_val = max(values[:-1])
    label_distance = max_val * 1.15
    
    for angle, label in zip(angles[:-1], display_labels):
        ax.text(angle, label_distance, label, 
                horizontalalignment='center', verticalalignment='center',
                fontsize=11, fontweight='normal')
    
    # Remove default tick labels
    ax.set_xticklabels([])
    
    # Set y-axis with clean formatting like reference
    ax.set_ylim(0, max_val * 1.3)
    
    # Create clean radial grid lines like in reference
    y_ticks = np.arange(0, max_val + 10, 20)  # Every 20%
    ax.set_yticks(y_ticks)
    ax.set_yticklabels([f'{int(tick)}%' for tick in y_ticks], fontsize=9, color='gray')
    
    # Clean title styling like reference image
    ax.set_title(title, size=16, fontweight='bold', pad=20, color='black')
    
    # Clean grid styling
    ax.grid(True, alpha=0.3, linestyle='-', linewidth=0.5, color='gray')
    ax.set_theta_zero_location('N')  # Start from top
    ax.set_theta_direction(-1)  # Clockwise
    
    # Remove radial axis label
    ax.set_ylabel('')
    
    # Remove legend since we only have one series
    
    plt.tight_layout()
    
    return fig, ax

In [ ]:
# Create the radar chart with styling similar to reference image
fig, ax = create_radar_chart(dx_counts, len(adult_df), 
                            title="Body Region Trauma Distribution")
plt.show()

In [ ]:
# save image
# fig.savefig("/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/analysis/adult_trauma/body_region_distribution.png", dpi=600, bbox_inches='tight')